In [1]:
import pickle
import numpy as np
# from sklearn import tree
# import catboost as cb
import xgboost as xgb
from tqdm.auto import tqdm

In [2]:
with open('X_out_train.pickle', 'rb') as f:
    X_train = pickle.load(f)
with open('Y_out_train.pickle', 'rb') as f:
    Y_train = pickle.load(f)
with open('X_out_val.pickle', 'rb') as f:
    X_val = pickle.load(f)
with open('Y_out_vaL.pickle', 'rb') as f:
    Y_val = pickle.load(f)

### SKlearn

In [10]:
# вектор y делю только на 0 и 1
mask = Y_train > 0.2  # все что > 0.2 - будет 1
Y_train_01 = np.zeros(len(Y_train))
Y_train_01[mask] = 1
# all_score_Y_train_01[~mask] = 0

In [ ]:
model_sklearn=tree.DecisionTreeClassifier(criterion="entropy")
model_sklearn.fit(X_train, Y_train_01)

### CatBoost

In [ ]:
model_cb = cb.CatBoostClassifier(task_type = "GPU", grow_policy='Depthwise',
                                iterations=1,
                                learning_rate=0.04,
                                max_depth=2,
                                eval_metric='RMSE',
                                metric_period = 20) # od_type and od_wait 
model_cb.fit(X_train, Y_train, 
             eval_set=(X_val, Y_val), 
             use_best_model=True)
print(model_cb.tree_count_)

In [ ]:
model_cb.save_model("model_cb_res.json", format="json")

### XGBoost

In [13]:
xgb_params = {'eta': 0.2, 
              'max_depth': 8, 
              'subsample': 0.6, 
              'colsample_bytree': 0.7, 
              'objective': 'binary:logistic', 
              'eval_metric': 'logloss', 
              }
num_rounds = 200
d_train = xgb.DMatrix(X_train, Y_train)
d_val = xgb.DMatrix(X_val, Y_val)
evallist = [(d_val, 'val')]

model_xgb = xgb.train(xgb_params, d_train, num_rounds, evallist, early_stopping_rounds=10)

[0]	val-logloss:0.51984
[1]	val-logloss:0.40371
[2]	val-logloss:0.32056
[3]	val-logloss:0.25914
[4]	val-logloss:0.21239
[5]	val-logloss:0.17635
[6]	val-logloss:0.14842
[7]	val-logloss:0.12656
[8]	val-logloss:0.10936
[9]	val-logloss:0.09582
[10]	val-logloss:0.08510
[11]	val-logloss:0.07669
[12]	val-logloss:0.07006
[13]	val-logloss:0.06477
[14]	val-logloss:0.06066
[15]	val-logloss:0.05745
[16]	val-logloss:0.05495
[17]	val-logloss:0.05297
[18]	val-logloss:0.05145
[19]	val-logloss:0.05027
[20]	val-logloss:0.04936
[21]	val-logloss:0.04862
[22]	val-logloss:0.04807
[23]	val-logloss:0.04764
[24]	val-logloss:0.04730
[25]	val-logloss:0.04705
[26]	val-logloss:0.04685
[27]	val-logloss:0.04668
[28]	val-logloss:0.04653
[29]	val-logloss:0.04642
[30]	val-logloss:0.04634
[31]	val-logloss:0.04626
[32]	val-logloss:0.04621
[33]	val-logloss:0.04615
[34]	val-logloss:0.04611
[35]	val-logloss:0.04607
[36]	val-logloss:0.04602
[37]	val-logloss:0.04601
[38]	val-logloss:0.04597
[39]	val-logloss:0.04595
[40]	val-l

In [15]:
with open('model_xgb_3_params.pickle', 'wb') as f:
    pickle.dump(xgb_params, f)
with open('model_xgb_3_pickle', "wb") as f:
    pickle.dump(model_xgb, f)
model_xgb.save_model('model_xgb_3.model')

## predict

In [16]:
with open('X_out_test.pickle', 'rb') as f:
    X_test = pickle.load(f)
with open('count_sent_in_text_X_test.pickle', 'rb') as f:
    count_sent_X_test = pickle.load(f)
with open('count_sent_in_text_Y_test.pickle', 'rb') as f:
    count_sent_Y_test = pickle.load(f)

In [2]:
my_model = xgb.Booster()
my_model.load_model('model_xgb_3.model')

In [5]:
dump_list = my_model.get_dump()
num_trees = len(dump_list)
print(num_trees)
my_model.get_score(importance_type='weight')

200


{'f45932': 177,
 'f22703': 1,
 'f2403': 1,
 'f46356': 1,
 'f24978': 1,
 'f21424': 583,
 'f39713': 87,
 'f33249': 22,
 'f22609': 1,
 'f19993': 2,
 'f7019': 1,
 'f46659': 3,
 'f12657': 393,
 'f26128': 7,
 'f10561': 1,
 'f38184': 130,
 'f28715': 2,
 'f29605': 2,
 'f12988': 1,
 'f39293': 209,
 'f25563': 5,
 'f39117': 66,
 'f28534': 97,
 'f43068': 11,
 'f48737': 1,
 'f626': 1,
 'f27043': 7,
 'f11230': 1,
 'f27056': 12,
 'f8902': 1,
 'f2210': 6,
 'f41312': 3,
 'f14417': 6,
 'f41754': 143,
 'f41560': 1,
 'f30736': 9,
 'f19054': 1,
 'f24444': 1,
 'f19591': 1,
 'f4817': 4,
 'f6008': 6,
 'f4406': 4,
 'f34407': 2,
 'f43007': 44,
 'f23470': 6,
 'f35741': 1,
 'f23683': 1,
 'f18918': 62,
 'f29990': 4,
 'f2108': 1,
 'f8507': 1,
 'f23110': 1,
 'f24055': 1,
 'f9858': 10,
 'f4801': 1,
 'f1736': 1,
 'f29166': 1,
 'f4155': 4,
 'f38188': 192,
 'f32503': 2,
 'f47815': 3,
 'f43876': 1,
 'f23174': 1,
 'f20312': 44,
 'f15011': 2,
 'f14287': 2,
 'f28963': 1,
 'f22209': 2,
 'f13733': 4,
 'f34615': 93,
 'f46012':

In [ ]:
d_test = xgb.DMatrix(X_test)
Y_predict = my_model.predict(d_test, iteration_range=(0, my_model.best_iteration))

In [12]:
print(X_test.shape)
print(Y_predict.shape)
print(max(Y_predict), min(Y_predict))

(217689, 50000)
(217689,)
0.26596063 0.00074877753


In [30]:
index_sent_for_summ = []
new_text_start = 0
for text_ind in tqdm(range(len(count_sent_X_test))):
    text_end = new_text_start + count_sent_X_test[text_ind]
    
    predict_for_one_text = Y_predict[new_text_start : text_end]
    sort_index = np.argsort(predict_for_one_text)[::-1] 
    index_sent_for_summ.append(sort_index[:count_sent_Y_test[text_ind]])  # беру лучшие (кол-во = кол-во в Y реальном)
    
    new_text_start = text_end  
    if text_ind%500==0:
        print(new_text_start, index_sent_for_summ[text_ind])

  0%|          | 0/5770 [00:00<?, ?it/s]

25 [20 18]
19101 [15  6 26  0]
38022 [ 1  6 21]
56568 [29 30 34]
75597 [4 6 9]
94343 [ 9 35  0  5]
112855 [ 5 15 21]
131907 [13 14 12]
150613 [33  0 24  7]
169600 [ 9  1 11]
188696 [21  2]
207704 [14]


In [31]:
with open('tree_index_sent_for_summ_X_test.pickle', 'wb') as f:
    pickle.dump(np.array(index_sent_for_summ), f)

<ipython-input-31-247669563042>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  pickle.dump(np.array(index_sent_for_summ), f)
